In [1]:
import pandas as pd

# intialise data of lists.
data = {'Name':['John', 'nick', 'krish', 'jack'],
        'Age':[20, 21, 19, 18],
        'GPA':[3.12, 2.98, 2.56, 3.68]}
 
# Create DataFrame
df = pd.DataFrame(data)

# Name the DataFrame
df.name = 'people'

# Print the output.
print(df)

    Name  Age   GPA
0   John   20  3.12
1   nick   21  2.98
2  krish   19  2.56
3   jack   18  3.68


In [2]:
import virus

In [3]:
virus.drop_table('spk')

In [3]:
%%time
import requests

def get_spk():
    # api-endpoint 
    URL = "https://ws.spk.gov.tr/PortfolioValues/api/PortfoyDegerleri/1/2010-10-01/2010-10-08"

    # Date format: YYYY-MM-DD
    

    # defining a params dict for the parameters to be sent to the API 
    #PARAMS = {'address':location} 

    # sending get request and saving the response as response object 
    #r = requests.get(url = URL, params = PARAMS) 
    r = requests.get(url = URL) 

    # extracting data in json format 
    data = r.json() 
    
    
    return data
    # printing the output 
    #print(data) 
    
a = get_spk()


    

CPU times: user 239 ms, sys: 47.1 ms, total: 286 ms
Wall time: 36.1 s


In [4]:
spk_df = pd.DataFrame(a)
spk_df = spk_df[['Tarih','FonKodu','BirimPayDegeri']]
spk_df['Tarih'] = pd.to_datetime(spk_df['Tarih']) 
spk_df.name = 'spk'


In [38]:
csv_s = spk_df.to_csv(index=False)

f = open("spk.csv", "w")
f.write(csv_s)
f.close()

In [6]:
%%time
virus.insert(spk_df)

-----------Table creation------------
CPU times: user 1.48 s, sys: 341 ms, total: 1.82 s
Wall time: 11 s


In [7]:
%%time
virus.insert_many(spk_df)

-----------Table creation------------
Get values in many: Time 0.02167 seconds
CPU times: user 56.2 ms, sys: 20.9 ms, total: 77.1 ms
Wall time: 411 ms


In [6]:
%%time
virus.insert_batch(spk_df)

-----------Table creation------------
Get values in many: Time 0.02065 seconds
CPU times: user 40.1 ms, sys: 8.02 ms, total: 48.2 ms
Wall time: 301 ms


In [9]:
%%time
virus.insert_batch_page(spk_df,100)

Get values in batch: Time 0.006443 seconds
CPU times: user 18 ms, sys: 1.2 ms, total: 19.2 ms
Wall time: 97.2 ms


In [5]:
%%time
virus.insert_copy(spk_df)

Get values in batch: Time 0.003456
Insert Error!
extra data after last expected column
CONTEXT:  COPY spk, line 1: "[Timestamp('2010-10-01 00:00:00'), 'AFO', 0.017706]|[Timestamp('2010-10-01 00:00:00'), 'AKU', 0.0399..."

CPU times: user 12.7 ms, sys: 7.37 ms, total: 20.1 ms
Wall time: 34.4 ms


In [21]:
import psycopg2
import psycopg2.extras
import numpy as np
import pandas as pd
import io
from typing import Iterator, Optional, Any
import time

connect_str = "dbname='algosis_db' user='tony' host='localhost' password='tony'"

def insert_copy(frame):

    #get table name
    table_name = frame.name

    row_count = frame.shape[0]

    try:
        # use our connection values to establish a connection
        conn = psycopg2.connect(connect_str)

        # create a psycopg2 cursor that can execute queries
        cursor = conn.cursor()

        virus.create_table(frame)
        
        start = time.perf_counter()
        
        variables = virus.get_values_efficient(frame);

        elapsed = time.perf_counter() - start
        print(f'Get values in batch: Time {elapsed:0.4}')

        csv_file_like_object = io.StringIO()

        #s = '|'.join(map(clean_csv_value,variables)) + '\n'
        s = "Timestamp('2010-10-01 00:00:00')|'AFO'|0.017706\nTimestamp('2010-10-01 00:00:00')|'AKU'|0.039933\n"
        print(s)
        
        csv_file_like_object.write(s)

        csv_file_like_object.seek(0)

        conn.commit() # <--- makes sure the change is shown in the database

    
        cursor.copy_from(csv_file_like_object, table_name, sep='|')

        cursor.close()
        conn.close()
    except Exception as e:

        print("Insert Error!")
        print(e)




def clean_csv_value(value: Optional[Any]) -> str:
    if value is None:
        return r'\N'
    return str(value).replace('\n', '\\n')


insert_copy(spk_df)

Get values in batch: Time 0.006254
'2010-10-01 00:00:00'|'AFO'|0.017706
'2010-10-01 00:00:00'|'AKU'|0.039933

Insert Error!
invalid input syntax for integer: "'2010-10-01 00:00:00'"
CONTEXT:  COPY spk, line 1, column id: "'2010-10-01 00:00:00'"



In [18]:
'|'.join(map(clean_csv_value, (
                2,
                3,
                4,
            ))) + '\n'

'2|3|4\n'

In [20]:
virus.get_values_s(spk_df)

{0: "('2010-10-01 00:00:00','AFO',0.017706)",
 1: "('2010-10-01 00:00:00','AKU',0.039933)",
 2: "('2010-10-01 00:00:00','ADP',0.076561)",
 3: "('2010-10-01 00:00:00','ADE',0.074587)",
 4: "('2010-10-01 00:00:00','AKE',0.0)",
 5: "('2010-10-01 00:00:00','AK3',1.134149)",
 6: "('2010-10-01 00:00:00','ANL',2.275574)",
 7: "('2010-10-01 00:00:00','APT',0.017632)",
 8: "('2010-10-01 00:00:00','AK2',10.975972)",
 9: "('2010-10-01 00:00:00','AOY',0.012755)",
 10: "('2010-10-01 00:00:00','AFT',0.010662)",
 11: "('2010-10-01 00:00:00','AED',0.010538)",
 12: "('2010-10-01 00:00:00','AYA',0.044156)",
 13: "('2010-10-01 00:00:00','AAK',12.488997)",
 14: "('2010-10-01 00:00:00','AAL',0.240097)",
 15: "('2010-10-01 00:00:00','IYB',0.297912)",
 16: "('2010-10-01 00:00:00','GTF',0.962344)",
 17: "('2010-10-01 00:00:00','GL1',15.757724)",
 18: "('2010-10-01 00:00:00','GMA',0.035276)",
 19: "('2010-10-01 00:00:00','GBL',0.049237)",
 20: "('2010-10-01 00:00:00','GBC',0.0)",
 21: "('2010-10-01 00:00:00','

In [17]:
virus.get('spk')

,id,tarih,fonkodu,birimpaydegeri
0,1,2010-10-01,AFO,0.017706
1,2,2010-10-01,AKU,0.039933
2,3,2010-10-01,ADP,0.076561
3,4,2010-10-01,ADE,0.074587
4,5,2010-10-01,AKE,0.000000
...,...,...,...,...
39046,39047,2011-09-30,HKV,43.722108
39047,39048,2011-09-30,HLL,0.024968
39048,39049,2011-09-30,TZD,0.076309
39049,39050,2011-09-30,ZBD,37.642544


In [10]:
%%time
virus.insert_sqlalchemy(spk_df)

(psycopg2.errors.UndefinedColumn) column "Tarih" of relation "spk" does not exist
LINE 1: INSERT INTO spk ("Tarih", "FonKodu", "BirimPayDegeri") VALUE...
                         ^
 [SQL: 'INSERT INTO spk ("Tarih", "FonKodu", "BirimPayDegeri") VALUES (%(Tarih)s, %(FonKodu)s, %(BirimPayDegeri)s)'] [parameters: ({'Tarih': datetime.datetime(2010, 10, 1, 0, 0), 'FonKodu': 'AFO', 'BirimPayDegeri': 0.017706}, {'Tarih': datetime.datetime(2010, 10, 1, 0, 0), 'FonKodu': 'AKU', 'BirimPayDegeri': 0.039933}, {'Tarih': datetime.datetime(2010, 10, 1, 0, 0), 'FonKodu': 'ADP', 'BirimPayDegeri': 0.076561}, {'Tarih': datetime.datetime(2010, 10, 1, 0, 0), 'FonKodu': 'ADE', 'BirimPayDegeri': 0.074587}, {'Tarih': datetime.datetime(2010, 10, 1, 0, 0), 'FonKodu': 'AKE', 'BirimPayDegeri': 0.0}, {'Tarih': datetime.datetime(2010, 10, 1, 0, 0), 'FonKodu': 'AK3', 'BirimPayDegeri': 1.134149}, {'Tarih': datetime.datetime(2010, 10, 1, 0, 0), 'FonKodu': 'ANL', 'BirimPayDegeri': 2.275574}, {'Tarih': datetime.datetime(

In [12]:
spk_df

,Tarih,FonKodu,BirimPayDegeri
0,2010-10-01,AFO,0.017706
1,2010-10-01,AKU,0.039933
2,2010-10-01,ADP,0.076561
3,2010-10-01,ADE,0.074587
4,2010-10-01,AKE,0.000000
...,...,...,...
3970,2010-11-10,HKV,42.466061
3971,2010-11-10,HLL,0.024223
3972,2010-11-10,TZD,0.093048
3973,2010-11-10,ZBD,36.672102


In [16]:
spk_df.dtypes[spk_df.columns[0]]

dtype('<M8[ns]')

In [17]:
virus.search()

,0
0,tutorials
1,spk
2,people


In [44]:
import psycopg2

connect_str = "dbname='algosis_db' user='tony' host='localhost' password='tony'"


def insert():

    try:
        # use our connection values to establish a connection
        conn = psycopg2.connect(connect_str)

        # create a psycopg2 cursor that can execute queries
        cursor = conn.cursor()

        
        #query_s = "INSERT INTO spk (Tarih,FonKodu,BirimPayDegeri) VALUES ('2005-10-10 00:00:00','AKU',0.021895);"
        
        query_s = "COPY spk (  Tarih,FonKodu,BirimPayDegeri )  FROM  'spk.csv' "
        
        """COPY table_name [ ( column [, ...] ) ]
    FROM { 'filename' | STDIN }
    [ [ WITH ] ( option [, ...] ) ]"""
        
        cursor.execute(query_s)
        
        conn.commit()
      
        cursor.close()
        conn.close()
    except Exception as e:

        print("Insert Error!")
        print(e)

insert()

Insert Error!
must be superuser to COPY to or from a file
HINT:  Anyone can COPY to stdout or from stdin. psql's \copy command also works for anyone.



In [46]:
import psycopg2

connect_str = "dbname='algosis_db' user='tony' host='localhost' password='tony'"


def alter_role():

    try:
        # use our connection values to establish a connection
        conn = psycopg2.connect(connect_str)

        # create a psycopg2 cursor that can execute queries
        cursor = conn.cursor()

        
        #query_s = "INSERT INTO spk (Tarih,FonKodu,BirimPayDegeri) VALUES ('2005-10-10 00:00:00','AKU',0.021895);"
        
        query_s = "ALTER USER tony WITH SUPERUSER "
 
        cursor.execute(query_s)
        
        conn.commit()
      
        cursor.close()
        conn.close()
    except Exception as e:

        print("Insert Error!")
        print(e)

alter_role()

Insert Error!
must be superuser to alter superusers



In [48]:
%%time
virus.get('spk')

CPU times: user 7.21 ms, sys: 0 ns, total: 7.21 ms
Wall time: 15.3 ms


,id,tarih,fonkodu,birimpaydegeri
0,1,2010-10-01,AFO,0.017706
1,2,2010-10-01,AKU,0.039933
2,3,2010-10-01,ADP,0.076561
3,4,2010-10-01,ADE,0.074587
4,5,2010-10-01,AKE,0.000000
...,...,...,...,...
846,847,2010-10-08,HKV,42.341170
847,848,2010-10-08,HLL,0.024151
848,849,2010-10-08,TZD,0.090625
849,850,2010-10-08,ZBD,36.493744


In [57]:
%%time
pd.DataFrame(a).columns

CPU times: user 10.9 ms, sys: 686 µs, total: 11.5 ms
Wall time: 10.4 ms


Index(['Tarih', 'FonKodu', 'FonUnvani', 'FonTipi', 'FonTuru', 'ToplamDeger',
       'BirimPayDegeri', 'DolasimdakiPaySayisi', 'YatirimciSayisi',
       'BankaBonosu', 'Diger', 'DevletTahvili', 'DovizOdemeliBono',
       'DovizOdemeliTahvil', 'Eurobond', 'FinansmanBonosu',
       'FonKatilmaBelgesi', 'GayrimenkulSertifikasi', 'HazineBonosu',
       'HisseSenedi', 'KamuDisBorclanmaAraci', 'KamuKiraSertifikasi',
       'KatilimHesabi', 'KiymetliMaden', 'OzelSektorKiraSertifikasi',
       'OzelSektorTahvili', 'TersRepo', 'TPP', 'TurevAraci',
       'VarligaDayaliMenkulKiymet', 'VadeliMevduat', 'YabanciBorclanmaAraci',
       'YabanciHisseSenedi', 'YabanciMenkulKiymet'],
      dtype='object')

In [67]:
main_cols = spk_df[['Tarih','FonKodu','BirimPayDegeri']]
#pd.to_datetime0 (main_cols)
import numpy as np
spk_df["date"] = pd.to_datetime( main_cols['Tarih'] )



In [65]:
spk_df.Tarih.apply(lambda x)

0      2005-10-10T00:00:00
1      2005-10-10T00:00:00
2      2005-10-10T00:00:00
3      2005-10-10T00:00:00
4      2005-10-10T00:00:00
              ...         
226    2005-10-12T00:00:00
227    2005-10-12T00:00:00
228    2005-10-12T00:00:00
229    2005-10-12T00:00:00
230    2005-10-12T00:00:00
Name: Tarih, Length: 231, dtype: object

In [72]:
spk_df.date.dtypes == '<M8[ns]'


True

In [75]:
(spk_df.date == pd.to_datetime(spk_df.date)).all()

True

In [83]:
spk_df.dtypes['Tarih'] == pd.to_datetime(spk_df['FonKodu'])

ParserError: Unknown string format: AKU

In [11]:
spk_df['Tarih'] == pd.to_datetime( spk_df['Tarih'] )

0      True
1      True
2      True
3      True
4      True
       ... 
226    True
227    True
228    True
229    True
230    True
Name: Tarih, Length: 231, dtype: bool

In [10]:
a= []
a.append('s')
a.append(1)
a

['s', 1]

In [11]:
type((1,2,3))

tuple